In [58]:
import numpy as np
import cvxpy as cp
import scipy as sp
from tabulate import tabulate

# Set the possible values of each bit: {0, 1}
a_range = b_range = x_range = y_range = range(2)


def generate_bitstrings(n: int) -> list:
    """
    Function that generates all possible bitstrings of length n.
    """
    if n == 0:
        return [[]]
    else:
        previous_bitstrings = generate_bitstrings(n - 1)
        current_bitstrings = []
        for bitstring in previous_bitstrings:
            current_bitstrings.append(bitstring + [0])
            current_bitstrings.append(bitstring + [1])
        return current_bitstrings

Here, the bitstring lengths that Alice and Bob receive can be set. $x=x\_range^m$ is for Alice, $y=y\_range^n$ is for Bob.

In [59]:
# These variables may be changed
m = 1
n = 1

# These variables shouldn't be changed
# Generates all possible bitstrings for x and y
x_values = generate_bitstrings(m)
y_values = generate_bitstrings(n)

$q(x,y)$ is the distribution function, which returns the probability of Alice receiving $x$ and Bob receiving $y$. For a uniform distribution, `q_uniform` can be called.

In [60]:
def q_uniform():
    x_possibilities = len(x_values)
    y_possibilities = len(y_values)

    return 1 / (x_possibilities * y_possibilities)


# This function may be changed
def q(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    :return: A user-specified probability
    """
    return q_uniform()

Alice and Bob win the game when $a \oplus b = f(x,y)$.
$f(x,y)$ can be set here.

In [61]:
# This function may be changed
def f(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    """
    return x[0] * y[0]

We rewrite the calculation of the entangled bias:

$$\varepsilon^*(G) = \sum_{xy} q(x,y)(-1)^{f(x,y)}\langle \Psi | A_x \otimes B_y | \Psi \rangle$$
$$\varepsilon^*(G) = \langle D, M \rangle$$

where
$$D(x,y)=q(x,y)(-1)^{f(x,y)}$$
and
$$M(x,y)=\langle \Psi | A_x \otimes B_y | \Psi \rangle$$

We will create the matrix $D$ here. $M$ is the matrix we want to optimize.

In [62]:
# This shouldn't be changed
def D_constructor(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    """
    return q(x, y) * ((-1) ** f(x, y))


D = np.matrix([[D_constructor(x, y) for y in y_values] for x in x_values])
print(D)

[[ 0.25  0.25]
 [ 0.25 -0.25]]


Here we start defining the semidefinite program. We start by creating the variables that need to be optimized.

From Tsirelson's theorem, remember

$$Z=\begin{pmatrix}
            R & M \\
            M^\dagger & S
        \end{pmatrix}
        \geq 0$$
where the diagonal entries of $R$ and $S$ are 1.

In [63]:
# Define the variables of the optimization problem
M = cp.Variable(D.shape)
R = cp.Variable((len(x_values), len(x_values)))
S = cp.Variable((len(y_values), len(y_values)))

# Create block matrix Z
Z = cp.bmat([[R, M], [cp.conj(M).T, S]])

Now, we add the constraints to the semidefinite program.

In [64]:
# Z is semidefinite
constraints = [Z >> 0]

# All diagonal entries of Z need to be 1.
constraints += [cp.diag(Z) == np.ones(Z.shape[0])]

Finally, we solve the optimization problem

$$\varepsilon^*(G) = \max \langle D, M \rangle$$
under the constraints defined above. Note that

$$\max \langle D, M \rangle = \max \mathrm{Tr}(D^\dagger M)$$

In [65]:
problem = cp.Problem(cp.Maximize(cp.trace(cp.conj(D).T @ M)),
                     constraints)
problem.solve()

# Print results.
print("The optimal entangled bias is", problem.value)
print("The entangled value is", 0.5 + problem.value / 2)

The optimal entangled bias is 0.7071084544426541
The entangled value is 0.853554227221327


We can also print the (now optimized) matrix $Z$.

In [66]:
table = tabulate(Z.value, tablefmt="simple_grid")
print(table)

┌──────────────┬──────────────┬──────────────┬──────────────┐
│  1           │ -1.94521e-15 │  0.707108    │  0.707108    │
├──────────────┼──────────────┼──────────────┼──────────────┤
│ -1.94521e-15 │  1           │  0.707108    │ -0.707108    │
├──────────────┼──────────────┼──────────────┼──────────────┤
│  0.707108    │  0.707108    │  1           │ -8.19023e-16 │
├──────────────┼──────────────┼──────────────┼──────────────┤
│  0.707108    │ -0.707108    │ -8.19023e-16 │  1           │
└──────────────┴──────────────┴──────────────┴──────────────┘


We will now reconstruct the sets of matrices $\{P_0^x\}$ and $\{Q_0^y\}$, which are the measurements Alice and Bob make on their qubit in the computational basis. Regarding the quantum state Alice and Bob are measuring, we know that they use a maximally entangled state:

$$|\Psi\rangle = \frac{1}{\sqrt{N}}\sum_{i=0}^{N-1}|i\rangle|i\rangle$$

The variables used in this equation can be found in the thesis in the section 'Recovering the strategy'.

We start the process by finding the Gram decomposition of matrix $M$. The columns of matrix $Q$ are the vectors which we will use in the next steps, for which holds: $M = Q^T Q$.

In [67]:
# Doesn't work when M isn't a square matrix

M_matrix = np.matrix(M.value, dtype='complex')
Q = np.matrix(sp.linalg.sqrtm(M_matrix))

We pick $N$ such that $2N+1\geq n+m$.
The Gram decomposition of a positive semidefinite matrix $G$ is a set of real vectors $u_1, \hdots, u_n, v_1, \hdots, v_m$. So $m,n$ are defined differently here.

In [68]:
import math

N = math.ceil((Q.shape[1] * 2 - 1.0) / 2.0)

Now we define the Weyl-Brauer and Pauli operators.

In [69]:
pauli_I = np.identity(2, dtype='complex')
pauli_sx = np.matrix([[0, 1], [1, 0]], dtype='complex')
pauli_sy = np.matrix([[0, complex(0, -1)], [complex(0, 1), 0]], dtype='complex')
pauli_sz = np.matrix([[1, 0], [0, -1]])


def wb_operator(i: int, order: int):
    x = None
    y = None
    z = None
    if i % 2 == 0:
        k = int(i / 2)
        if k - 1 != 0:
            z = pauli_sz
            for _ in range(1, k - 1):
                z = np.kron(z, pauli_sz)
            y = np.kron(z, pauli_sy)
        else:
            y = pauli_sy

        for _ in range(0, order - k):
            y = np.kron(y, pauli_I)
        return y

    elif i == 2 * order + 1:
        z = pauli_sz
        for _ in range(1, order):
            z = np.kron(z, pauli_sz)
        return z

    else:
        k = int((i + 1) / 2)
        if k - 1 != 0:
            z = pauli_sz
            for _ in range(1, k - 1):
                z = np.kron(z, pauli_sz)
            x = np.kron(z, pauli_sy)
        else:
            x = pauli_sy
        for _ in range(0, order - k):
            x = np.kron(x, pauli_I)
        return x


We can now reconstruct the sets of operators $A_x, B_y$.

In [71]:
A = np.zeros((Q.shape[1], *wb_operator(1, N).shape), dtype='complex')
B = np.zeros((Q.shape[1], *wb_operator(1, N).shape), dtype='complex')

for j in range(Q.shape[1]):
    for i in range(1, 2 * Q.shape[1]):
        A[j] += B[i, j] * wb_operator(i, N)

for k in range(Q.shape[1]):
    for i in range(1, 2 * Q.shape[1]):
        B[k] += B[i, k] * wb_operator(i, N).T

print(A)
print(B)

x
I
x
I
x
I


IndexError: index 2 is out of bounds for axis 0 with size 2